In [1]:
#import all necessary libaries

from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

In [2]:
checkpoint_directory = './model/training_checkpoints'

test_fp  = "./training/test_dataset.csv"

In [3]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(5,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(5)
])
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
status = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_directory))

In [4]:
def pack_features_vector(features, labels):
  """Pack the features into a single array."""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels

In [5]:
batch_size = 32
column_names = ['humidity', 'temperature', 'pm2_5', 'pm10', 'asthmatic_level',
                'predicted_comfort_level']

test_dataset = tf.data.experimental.make_csv_dataset(
    test_fp,
    batch_size, 
    column_names=column_names,
    label_name='predicted_comfort_level',
    num_epochs=1,
    shuffle=False)

test_dataset = test_dataset.map(pack_features_vector)

In [6]:
test_accuracy = tfe.metrics.Accuracy()

for (x, y) in test_dataset:
  logits = model(x)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracy(prediction, y)

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

Test set accuracy: 74.174%


In [7]:
# class_names = ['Very Good', 'Ok', 'Uncomfortable', 'Very Uncomfortable', 'Hazardous']

# humidity = 0.14
# temperature = 0.8
# pm2_5 = 1.91
# pm10 = 1.09
# is_asthmatic = 0.1

# logits = [humidity,temperature,pm2_5,pm10,is_asthmatic]
# class_idx = tf.argmax(logits).numpy()
# p = tf.nn.softmax(logits)[class_idx]
# name = class_names[class_idx]
# print("class id: " + str(class_idx))
# print("Example prediction: {} ({:4.1f}%)".format(name, 100*p))